# Use Stripe with agents.json
This notebook demonstrates how to use agents.json to interact with the Stripe API. This simple example shows loading the agents.json file and using it to execute Stripe tasks.


In [1]:
%pip install agentsjson
%pip install openai


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Setup API keys and references to `agents.json` files

In [ ]:
ROOTLY_API_KEY="your_rootly_api_key"
OPENAI_API_KEY="your_openai_api_key"
agents_json_url = "https://gist.githubusercontent.com/yagnyaPatel/f21f8c0a6c221213584bbb7a99a0e1dc/raw/bbfbe41c80042722ee65602d7b839a7787fa6007/agents.json"

### Load the agents.json file

In [5]:
from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
import agentsjson.core as core

# load the agents.json file
data: Bundle = core.load_agents_json(agents_json_url)
flows = data.agentsJson.flows

### Define the Agent

In [6]:
query = "Get me the incident details on the prod outage"

In [7]:
from agentsjson.core import ToolFormat

# Format the flows data for the prompt
flows_context = core.flows_prompt(flows)

# Create the system prompt
system_prompt = f"""You are an AI assistant that helps users interact with the Rootly API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

### Configure Auth

In [8]:
from agentsjson.core.models.auth import AuthType, BearerAuthConfig
auth = BearerAuthConfig(type=AuthType.BEARER, token=ROOTLY_API_KEY)

### Run the agent

In [9]:
from openai import OpenAI
from agentsjson.core.executor import execute_flows
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    tools=core.flows_tools(flows, format=ToolFormat.OPENAI),
    temperature=0
)

response = execute_flows(response, format=core.ToolFormat.OPENAI, bundle=data, flows=flows, auth=auth)

response

ModuleNotFoundError: No module named 'agentsjson.integrations.rootly'